# Imports

In [2]:
import numpy as np

# Forward Filtering

In [84]:
rows = 3 # amount of rows of the grid
columns = 3 # amount of columns of the grid

start_probability = 1/(rows * columns * 4)
f_old = np.array([[start_probability] for x in range((rows * columns * 4))])



In [91]:
transition_matrix(rows,columns)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [90]:
def transition_matrix(rows,columns):
    '''
    calculates the transistion matrix
    
    '''
    rows_transition = rows*columns*4
    columns_transition = rows_transition
    T = np.zeros((rows_transition,columns_transition))

    for n in range(rows_transition):
        t = n % 4 # gives the heading t = 0: north t = 1 East t = 2 South T=3 West
        direction_change = False
        headings = [False,False,False,False]
        walls = 0
        if n-4*rows < 0:
            headings[0] =  0
            walls += 1
            if t == 0:
                direction_change = True
        if n%(4*columns)+4  >= 4*columns:
            headings[1] = 0
            walls += 1
            if t == 1:
                direction_change = True
        if (n-t)+4*rows > 4*rows*columns:
            headings[2] = 0
            walls += 1
            if t == 2:
                direction_change = True
        
        if n%(4*columns)-4 < 0:
            headings[3] = 0
            walls += 1
            if t == 3:
                direction_change = True
        

        for i,e in enumerate(headings):
            if direction_change:
                if e is False:
                    headings[i] = 1/(4-walls)
            elif e is False: 
                if i == t:
                    headings[i] = 0.7
                else:
                    headings[i] = 0.3/(3-walls)

        #north           
        try:
            T[n,(n-t)-4*rows ] =  headings[0] 
        except IndexError:
            continue
        #east
        try:
            T[n,(n-t+1) +4] = headings[1]
        except IndexError:
            continue
        #south
        try:
            T[n,(n-t+2)+4*rows] = headings[2] 
        except IndexError:
            continue
        #west
        try:
            T[n,(n-t+3)-4] = headings[3]
        except IndexError:
            continue
    

        
    return T
            
            


In [190]:
observation_matrix(4,4)

[[ 0.625  0.5    0.5    0.625]
 [ 0.5    0.325  0.325  0.5  ]
 [ 0.5    0.325  0.325  0.5  ]
 [ 0.625  0.5    0.5    0.625]]


array([[ 0.625,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.625,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.625, ...,  0.   ,  0.   ,  0.   ],
       ..., 
       [ 0.   ,  0.   ,  0.   , ...,  0.625,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.625,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.625]])

In [165]:
def distance(start,end):
    max = abs(start[0]-end[0])
    if abs(start[1]-end[1]) > max:
        max = abs(start[1]-end[1])
    return max

In [168]:
print(distance((1,2),(1,5)))

3


In [200]:
def observation_matrix(e, rows, columns):
    rows_observation = rows*columns*4
    columns_observation = rows_observation
    O = []
    for m in range(rows):
        for n in range(columns):
            O_i = np.zeros((rows,columns))
            
            Observationmatrix = np.zeros((rows_observation,columns_observation))
            for k in range(rows):
                for l in range(columns):
                    index = 4*k*columns+4*l
                    if m==k and n==l:
                        O_i[k,l] = 0.1
                        for t in range(4):
                            Observationmatrix[index+t,index+t] = 0.1
                    elif distance((m,n),(k,l)) == 1:
                        O_i[k,l] = 0.05
                        for t in range(4):
                            Observationmatrix[index+t,index+t] = 0.05
                    elif distance((m,n),(k,l)) == 2:
                        O_i[k,l] = 0.025
                        for t in range(4):
                            Observationmatrix[index+t,index+t] = 0.025
            #print(O_i)   
        
            O.append(Observationmatrix)
    O_i = np.zeros((rows,columns))
    for k in range(rows):
        for l in range(columns):
            index = 4*k*columns+4*l
            if l == 0 or l== (columns-1):
                if k == 0 or k == (rows-1):
                    O_i[k,l] = 0.625
                    for t in range(4):
                        Observationmatrix[index+t,index+t] = 0.625
                else:
                    O_i[k,l] = 0.5
                    for t in range(4):
                        Observationmatrix[index+t,index+t] = 0.5
            elif k == 0 or k == (rows-1):
                O_i[k,l] = 0.5
                for t in range(4):
                    Observationmatrix[index+t,index+t] = 0.5
            else:
                O_i[k,l] = 0.325
                for t in range(4):
                    Observationmatrix[index+t,index+t] = 0.325
    #print(O_i)
    O.insert(0, Observationmatrix)
                
                    
                
    
    right_list = e[0]*columns + e[1]*4 + 1
    return O[right_list]
        
        
        

    

In [201]:
m = 4
n = 4
dimension = m * n * 4
probability = 1/(dimension)
f_old = np.array([[probability] for y in range(dimension)])
T = transition_matrix(m,n)
e = (0,0)
print(forward_filtering(e,m,n,f_old,T))

[[ 0.4198911 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.4198911 ]
 [ 0.15745916]
 [ 0.31491833]
 [ 0.        ]
 [ 0.20994555]
 [ 0.07872958]
 [ 0.10497278]
 [ 0.        ]
 [ 0.15745916]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.20994555]
 [ 0.        ]
 [ 0.31491833]
 [ 0.15745916]
 [ 0.15745916]
 [ 0.20994555]
 [ 0.20994555]
 [ 0.15745916]
 [ 0.07872958]
 [ 0.07872958]
 [ 0.10497278]
 [ 0.10497278]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.10497278]
 [ 0.        ]
 [ 0.10497278]
 [ 0.07872958]
 [ 0.10497278]
 [ 0.10497278]
 [ 0.07872958]
 [ 0.07872958]
 [ 0.10497278]
 [ 0.07872958]
 [ 0.07872958]
 [ 0.10497278]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [194]:
def forward_filtering(e,n,m,f_old,T):
    '''
    description

    :param e: the current evidence for time step t (position)
    '''
    
    
    f_new = np.dot(np.dot(observation_matrix(e,m,n), T.T),f_old)
    f_normalization = f_new / np.linalg.norm(f_new)
    

    


    return f_normalization